In [ ]:
import flask
app = flask.Flask(__name__)

#-------- MODEL GOES HERE -----------#
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


#-------- ROUTES GO HERE -----------#
@app.route('/')
def page():
    with open("RecommenderLanding.html", 'r') as viz_file:
        return viz_file.read()

@app.route('/result', methods=['POST'])
def result():
    '''Gets prediction using the HTML form'''
    # if flask.request.method == 'POST':
    df = pd.read_csv('df.csv', encoding = "ISO-8859-1")
    inputs = flask.request.form

    input1 = inputs['tenure(months)']
    input2 = 1
    input3 = inputs['gender']
    input4 = inputs['age']
    input5 = inputs['industry']
    input6 = inputs['profession']
    input7 = inputs['extraversion']
    input8 = inputs['independ']
    input9 = inputs['selfcontrol']
    input10 = inputs['anxiety']
    input11 = inputs['novator']
    input12 = float(inputs['weigh'])
    input_df = pd.DataFrame()
    input_df = pd.DataFrame([[input1,input2,input3,input4,input5,input6,input7,input8,input9,input10,input11]]
                            ,columns=['tenure', 'left_company', 'gender', 'age', 'industry', 'profession',
           'extraversion', 'independ', 'selfcontrol', 'anxiety', 'novator'])

    df_new = df.append(input_df).reset_index(drop=True)
    df_numeric = df_new.drop(['left_company','gender','industry','profession'], axis=1)
    df_dummies = pd.get_dummies(df_new['gender'], drop_first=True)

    ss = StandardScaler().fit(df_numeric)
    df_numeric_ss = ss.transform(df_numeric)
    df_numeric_ss = pd.DataFrame(df_numeric_ss, columns = df_numeric.columns.values)
    df_clean = pd.concat([df_numeric_ss, df_dummies], axis=1)
    soft = ['extraversion','independ','selfcontrol','anxiety','novator']
    df_clean_others = df_clean.drop(soft, axis=1)
    df_clean_soft = df_clean[soft]

    df_clean_soft_reweigh = df_clean_soft*input12
    df_clean_others_reweigh = df_clean_others*(1-input12)

    df_clean2 = pd.concat([df_clean_soft_reweigh,df_clean_others_reweigh], axis=1)
    data_matrix2 = pd.DataFrame.as_matrix(df_clean2)

    cosine_sim2 = cosine_similarity(data_matrix2, data_matrix2)

    # returnString3 = '<p>ind1:{}</p><p>prof1:{}</p><p>ind2:{}</p><p>prof2:{}</p><p>ind3:{}</p><p>prof3:{}</p><p>'.format(ind1,prof1,ind2,prof2,ind3,prof3)
    df_new = df.iloc[pd.DataFrame(cosine_sim2[1129])[0].sort_values(ascending=False).index[1:]]
    df_new['industry'] = df['industry'].apply(lambda x: 'Others' if x=='etc' or x==' HoReCa' else x)
    df_new['profession'] = df['profession'].apply(lambda x: 'Others' if x=='etc' or x==' HoReCa' else x)
    df_rec = df_new[df_new['left_company'] != 1]
    rec_prof = df_rec['profession'].unique()[:3]
    rec_indus = df_rec['industry'].unique()[:3]

    df_non_rec = df_new[(df_new['left_company'] == 1) & (df_new['tenure'] < 12)]
    nonrec_prof = df_non_rec['profession'].unique()[:3]
    nonrec_indus = df_non_rec['industry'].unique()[:3]

    # returnString3 = '<p>rec_prof:{}</p><p>rec_indus:{}</p><p>'.format(rec_prof,rec_indus)
    returnString3 = '<p>rec_prof:{}</p><p>rec_indus:{}</p><p>non_rec_prof:{}</p><p>non_rec_indus:{}</p><p>'.format(rec_prof,rec_indus,nonrec_prof,nonrec_indus)

    return returnString3


if __name__ == '__main__':
    '''Connects to the server'''

    HOST = '127.0.0.1'
    PORT = 4000

    app.run(HOST, PORT)
